# Academic abstract collection

In [1]:
# import sys
# !{sys.executable} -m pip install nltk
# # !{sys.executable} -m pip install wordcloud
from bs4 import BeautifulSoup
import requests
from bs4 import NavigableString
import queue
from concurrent.futures import ThreadPoolExecutor
import itertools
import threading
import csv
import pandas as pd 
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import numpy as np
import re
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

## Downfree html links

### Function (1)
**Return a list with a link to each paper**
* NB: This is for a single page

In [2]:
def get_links(soup):  
    
    papers = []
    for paper in soup.findAll("li",class_="list-group-item downfree"):
        papers.append("https://ideas.repec.org"+paper.a.get('href'))
                
    return(papers)

### Function (2)
**Soup the necessary URL and call Function (1) to return the list of links**

In [3]:
def parse_links(START_URL):
    
    page   = requests.get(START_URL)
    soup   = BeautifulSoup(page.text, 'html.parser')
    links = get_links(soup)

    return links

In [4]:
links = parse_links("https://ideas.repec.org/s/aea/aecrev.html") # American Economic Review
links.extend(parse_links("https://ideas.repec.org/s/aea/aecrev2.html"))
links.extend(parse_links("https://ideas.repec.org/s/aea/aecrev3.html"))
links.extend(parse_links("https://ideas.repec.org/s/aea/aecrev4.html"))
links.extend(parse_links("https://ideas.repec.org/s/aea/aecrev5.html"))
links.extend(parse_links("https://ideas.repec.org/s/aea/aecrev6.html"))
links.extend(parse_links("https://ideas.repec.org/s/aea/aecrev7.html"))
links.extend(parse_links("https://ideas.repec.org/s/aea/aecrev8.html"))
links.extend(parse_links("https://ideas.repec.org/s/aea/aecrev9.html"))
links.extend(parse_links("https://ideas.repec.org/s/aea/aecrev10.html"))
links.extend(parse_links("https://ideas.repec.org/s/aea/aecrev11.html"))
links.extend(parse_links("https://ideas.repec.org/s/aea/aecrev12.html"))
links.extend(parse_links("https://ideas.repec.org/s/aea/aecrev13.html"))
links.extend(parse_links("https://ideas.repec.org/s/aea/aecrev14.html"))
links.extend(parse_links("https://ideas.repec.org/s/aea/aecrev15.html"))
links.extend(parse_links("https://ideas.repec.org/s/aea/aecrev16.html"))
links.extend(parse_links("https://ideas.repec.org/s/aea/aecrev17.html"))
links.extend(parse_links("https://ideas.repec.org/s/aea/aecrev18.html"))
links.extend(parse_links("https://ideas.repec.org/s/aea/aecrev19.html"))
links.extend(parse_links("https://ideas.repec.org/s/aea/aecrev20.html"))
links.extend(parse_links("https://ideas.repec.org/s/aea/aecrev21.html"))
links.extend(parse_links("https://ideas.repec.org/s/aea/aecrev22.html"))

aejmac = len(links)

In [5]:
print("aejmac",aejmac)

aejmac 2160


In [8]:
def get_abstracts(URL):
    
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    
    abstracts = []
    
    page      = session.get(URL)
    soup      = BeautifulSoup(page.text, 'html.parser')
    
    try:
    
        abstract  = soup.find('div', id='abstract-body').get_text()
    
    except:
        pass
    
    try:
        
        info      = [desc.strip() for desc in 
                     soup.find("li",class_="list-group-item downfree").descendants if type(desc) == NavigableString]
        
        abstracts.append({'Author/s': info[0],'Year': info[0].split()[-2], 
                      'Title': info[1], 'ID': info[3], 'Abstract': abstract})
        
    except:
        pass
    
    return abstracts

In [9]:
with ThreadPoolExecutor(20) as pool:
      abstracts = pool.map(get_abstracts, links)

aca_abstracts = list(abstracts)

with open('top5_data_resub.csv', 'w', newline='') as csvfile:
    fieldnames = aca_abstracts[0][0].keys()
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for x in range(0,len(aca_abstracts)):
        for row in aca_abstracts[x]:
            writer.writerow(row)

## Downgate html links

In [13]:
def get_links(soup):  
    
    papers = []
    for paper in soup.findAll("li",class_="list-group-item downgate"): #downgate here not downfree
        papers.append("https://ideas.repec.org"+paper.a.get('href'))
                
    return(papers)

In [14]:
def parse_links(START_URL):
    
    page   = requests.get(START_URL)
    soup   = BeautifulSoup(page.text, 'html.parser')
    links = get_links(soup)

    return links

In [15]:
links = parse_links("https://ideas.repec.org/s/ucp/jpolec.html") # Journal of Political Economy
links.extend(parse_links("https://ideas.repec.org/s/ucp/jpolec2.html"))
links.extend(parse_links("https://ideas.repec.org/s/ucp/jpolec3.html"))
links.extend(parse_links("https://ideas.repec.org/s/ucp/jpolec4.html"))
links.extend(parse_links("https://ideas.repec.org/s/ucp/jpolec5.html"))

jpolec = len(links) 

In [16]:
links.extend(parse_links("https://ideas.repec.org/s/oup/qjecon.html")) # The Quarterly Journal of Economics
links.extend(parse_links("https://ideas.repec.org/s/oup/qjecon2.html"))
links.extend(parse_links("https://ideas.repec.org/s/oup/qjecon3.html"))
links.extend(parse_links("https://ideas.repec.org/s/oup/qjecon4.html"))
links.extend(parse_links("https://ideas.repec.org/s/oup/qjecon5.html"))

qjecon = len(links) - jpolec

In [17]:
links.extend(parse_links("https://ideas.repec.org/s/oup/restud.html")) # Review of Economic Studies
links.extend(parse_links("https://ideas.repec.org/s/oup/restud2.html"))
links.extend(parse_links("https://ideas.repec.org/s/oup/restud3.html"))
links.extend(parse_links("https://ideas.repec.org/s/oup/restud4.html"))
links.extend(parse_links("https://ideas.repec.org/s/oup/restud5.html"))

restud = len(links) - jpolec - qjecon

In [18]:
links.extend(parse_links("https://ideas.repec.org/s/eee/jfinec.html")) # Journal of Finance
links.extend(parse_links("https://ideas.repec.org/s/eee/jfinec2.html"))
links.extend(parse_links("https://ideas.repec.org/s/eee/jfinec3.html"))
links.extend(parse_links("https://ideas.repec.org/s/eee/jfinec4.html"))
links.extend(parse_links("https://ideas.repec.org/s/eee/jfinec5.html"))
links.extend(parse_links("https://ideas.repec.org/s/eee/jfinec6.html"))
links.extend(parse_links("https://ideas.repec.org/s/eee/jfinec7.html"))
links.extend(parse_links("https://ideas.repec.org/s/eee/jfinec8.html"))
links.extend(parse_links("https://ideas.repec.org/s/eee/jfinec9.html"))
links.extend(parse_links("https://ideas.repec.org/s/eee/jfinec10.html"))
links.extend(parse_links("https://ideas.repec.org/s/eee/jfinec11.html"))

jfinec = len(links) - jpolec - qjecon - restud

In [19]:
print("jpolec:",jpolec,"qjecon:",qjecon,"restud",restud,"jfinec",jfinec)
print()
print("Total:",len(links))

jpolec: 1001 qjecon: 1001 restud 1001 jfinec 2201

Total: 5204


In [20]:
def get_abstracts(URL):
    
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    
    abstracts = []
    
    page      = session.get(URL)
    soup      = BeautifulSoup(page.text, 'html.parser')
    
    try:
    
        abstract  = soup.find('div', id='abstract-body').get_text()
    
    except:
        pass
    
    try:
        
        info      = [desc.strip() for desc in 
                     soup.find("li",class_="list-group-item downgate").descendants if type(desc) == NavigableString]
        
        abstracts.append({'Author/s': info[0],'Year': info[0].split()[-2], 
                      'Title': info[1], 'ID': info[3], 'Abstract': abstract})
        
    except:
        pass
    
    return abstracts

In [21]:
with ThreadPoolExecutor(20) as pool:
      abstracts = pool.map(get_abstracts, links)

aca_abstracts = list(abstracts)

with open('top5_data_resub_2.csv', 'w', newline='') as csvfile:
    fieldnames = aca_abstracts[0][0].keys()
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for x in range(0,len(aca_abstracts)):
        for row in aca_abstracts[x]:
            writer.writerow(row)

In [ ]:
# links.extend(parse_links("https://ideas.repec.org/s/ucp/jlabec.html")) #Journal of labour economics
# links.extend(parse_links("https://ideas.repec.org/s/ucp/jlabec2.html"))
# links.extend(parse_links("https://ideas.repec.org/s/ucp/jlabec3.html"))

In [ ]:
# links.extend(parse_links("https://ideas.repec.org/s/eee/jfinec.html")) # Journal of Financial Economics
# links.extend(parse_links("https://ideas.repec.org/s/eee/jfinec1.html"))
# links.extend(parse_links("https://ideas.repec.org/s/eee/jfinec2.html"))
# links.extend(parse_links("https://ideas.repec.org/s/eee/jfinec3.html"))
# links.extend(parse_links("https://ideas.repec.org/s/eee/jfinec4.html"))
# links.extend(parse_links("https://ideas.repec.org/s/eee/jfinec5.html"))
# links.extend(parse_links("https://ideas.repec.org/s/eee/jfinec6.html"))
# links.extend(parse_links("https://ideas.repec.org/s/eee/jfinec7.html"))
# links.extend(parse_links("https://ideas.repec.org/s/eee/jfinec8.html"))
# links.extend(parse_links("https://ideas.repec.org/s/eee/jfinec9.html"))
# links.extend(parse_links("https://ideas.repec.org/s/eee/jfinec10.html"))

In [ ]:
# links.extend(parse_links("https://ideas.repec.org/s/eee/moneco.html")) # Journal of Monetary Economics
# links.extend(parse_links("https://ideas.repec.org/s/eee/moneco2.html"))
# links.extend(parse_links("https://ideas.repec.org/s/eee/moneco3.html"))
# links.extend(parse_links("https://ideas.repec.org/s/eee/moneco4.html"))
# links.extend(parse_links("https://ideas.repec.org/s/eee/moneco5.html"))
# links.extend(parse_links("https://ideas.repec.org/s/eee/moneco6.html"))
# links.extend(parse_links("https://ideas.repec.org/s/eee/moneco7.html"))
# links.extend(parse_links("https://ideas.repec.org/s/eee/moneco8.html"))